## Project Overview
In this project, I scrape data from IMDb's top 1000 movies of all time list and create a dataset.

### Step 1: Get necessary libraries

In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep

### Step 2: Iterate and parse all pages 
The list contains 1000 movies with 10 pages containing 100 movies each. 

In [2]:
# create an object containing the dynamic part of the link as you go from one page to another
links = np.arange(1, 1000, 100)
links

array([  1, 101, 201, 301, 401, 501, 601, 701, 801, 901])

In [3]:
# create empty variable list to save extracted data
movie_data = []

In [4]:
# iterate through each page
for link in links:
    # fetch html content from urls
    link = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(link)+"&ref_=adv_nxt")
    # parse pulled html content
    soup = BeautifulSoup(link.content, 'html.parser')
    # create object to pull individual movie info
    movie_info = soup.find_all('div', class_ = 'lister-item mode-advanced')
    # add delay of 5 seconds
    sleep(5)
    # extract individual movie variables for all movies
    for movie in movie_info:
        # movie rank
        rank = movie.find('span', class_ = 'lister-item-index').text
        # movie title
        title = movie.find('h3', class_ = 'lister-item-header').a.text
        # release year
        year = movie.find('span', class_ = 'lister-item-year').text
        # run time
        time = movie.find('span', class_ = 'runtime').text
        # rating or leave empty if no value
        rate = movie.find('span', class_ = 'certificate').text if movie.find('span', class_ = 'certificate') else ''
        # star ratings
        stars = movie.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
        # metascore or leave empty if no value
        score = movie.find('span', class_ = 'metascore').text.replace(' ','') if movie.find('span', class_ = 'metascore') else ''  
        # votes and gross have the same tag; create variable containing both
        vote_gross = movie.find_all('span', attrs = {'name': 'nv'})
        # extract number of votes from vote_gross
        vote = vote_gross[0].text.strip()
        # extract gross revenue from vote_gross and leave empty if no value
        revenue = vote_gross[1].text if len(vote_gross) > 1 else ''

        # add all variables to empty list created earlier
        movie_data.append((rank, title, year, time, rate, stars, score, vote, revenue))

### Step 3: Create dataframe of extracted data

In [5]:
movie_df = pd.DataFrame(movie_data,
                 # add column names
                 columns = ['Rank',
                            'Movie Title',
                            'Year of Release',
                            'Run Time (minutes)',
                            'Rating',
                            'Star Rating',
                            'Metascore',
                            'Number of Votes',
                            'Gross Revenue in millions (USD)'
                           ])

# overview of the dataframe
movie_df.head(50)

,Rank,Movie Title,Year of Release,Run Time (minutes),Rating,Star Rating,Metascore,Number of Votes,Gross Revenue in millions (USD)
0,1.,The Shawshank Redemption,(1994),142 min,R,9.3,82,"2,796,722",$28.34M
1,2.,The Godfather,(1972),175 min,R,9.2,100,"1,948,395",$134.97M
2,3.,The Dark Knight,(2008),152 min,PG-13,9.0,84,"2,777,605",$534.86M
3,4.,Schindler's List,(1993),195 min,R,9.0,95,"1,406,133",$96.90M
4,5.,The Lord of the Rings: The Return of the King,(2003),201 min,PG-13,9.0,94,"1,915,094",$377.85M
5,6.,The Godfather Part II,(1974),202 min,R,9.0,90,"1,323,371",$57.30M
6,7.,12 Angry Men,(1957),96 min,Approved,9.0,97,"831,117",$4.36M
7,8.,Pulp Fiction,(1994),154 min,R,8.9,95,"2,145,290",$107.93M
8,9.,Inception,(2010),148 min,PG-13,8.8,74,"2,465,944",$292.58M
9,10.,The Lord of the Rings: The Fellowship of the Ring,(2001),178 min,PG-13,8.8,92,"1,943,254",$315.54M


### Step 4. Clean up dataset

In [6]:
# all columns are in the data type 'object' and no missing values 
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Rank                             1000 non-null   object
 1   Movie Title                      1000 non-null   object
 2   Year of Release                  1000 non-null   object
 3   Run Time (minutes)               1000 non-null   object
 4   Rating                           1000 non-null   object
 5   Star Rating                      1000 non-null   object
 6   Metascore                        1000 non-null   object
 7   Number of Votes                  1000 non-null   object
 8   Gross Revenue in millions (USD)  1000 non-null   object
dtypes: object(9)
memory usage: 70.4+ KB


In [7]:
# view descriptive statistics of all columns
movie_df.describe()

,Rank,Movie Title,Year of Release,Run Time (minutes),Rating,Star Rating,Metascore,Number of Votes,Gross Revenue in millions (USD)
count,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,1000,994,123,142,17,17,61,1000,736
top,1.,Beauty and the Beast,(2004),130 min,R,7.7,,"2,796,722",
freq,1,2,28,24,357,174,154,1,163


In [8]:
# skim rank column
movie_df['Rank'].unique()

array(['1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.', '11.',
       '12.', '13.', '14.', '15.', '16.', '17.', '18.', '19.', '20.',
       '21.', '22.', '23.', '24.', '25.', '26.', '27.', '28.', '29.',
       '30.', '31.', '32.', '33.', '34.', '35.', '36.', '37.', '38.',
       '39.', '40.', '41.', '42.', '43.', '44.', '45.', '46.', '47.',
       '48.', '49.', '50.', '51.', '52.', '53.', '54.', '55.', '56.',
       '57.', '58.', '59.', '60.', '61.', '62.', '63.', '64.', '65.',
       '66.', '67.', '68.', '69.', '70.', '71.', '72.', '73.', '74.',
       '75.', '76.', '77.', '78.', '79.', '80.', '81.', '82.', '83.',
       '84.', '85.', '86.', '87.', '88.', '89.', '90.', '91.', '92.',
       '93.', '94.', '95.', '96.', '97.', '98.', '99.', '100.', '101.',
       '102.', '103.', '104.', '105.', '106.', '107.', '108.', '109.',
       '110.', '111.', '112.', '113.', '114.', '115.', '116.', '117.',
       '118.', '119.', '120.', '121.', '122.', '123.', '124.', '125.',
       '12

In [9]:
# remove period after rank number and convert column to integer
movie_df['Rank'] = movie_df["Rank"].str.extract('(\d+)', expand = False).astype('int')
# view first 5 rows
movie_df['Rank'][:5]

0    1
1    2
2    3
3    4
4    5
Name: Rank, dtype: int64

In [10]:
# skim year of release column
movie_df['Year of Release'].unique()

array(['(1994)', '(1972)', '(2008)', '(1993)', '(2003)', '(1974)',
       '(1957)', '(2010)', '(2001)', '(1999)', '(1966)', '(2002)',
       '(2021)', '(2022)', '(2023)', '(2014)', '(1990)', '(1975)',
       '(1980)', '(2020)', '(1995)', '(1991)', '(1998)', '(1977)',
       '(1997)', '(1946)', '(1954)', '(1962)', '(2000)', '(2006)',
       '(1979)', '(2019)', '(1985)', '(2012)', '(1960)', '(1942)',
       '(2011)', '(1988)', '(1968)', '(1936)', '(1931)', '(II) (2018)',
       '(I) (2019)', '(1981)', '(2018)', '(1984)', '(1986)', '(I) (2017)',
       '(2016)', '(2009)', '(1964)', '(1950)', '(1940)', '(1963)',
       '(2004)', '(1983)', '(1971)', '(1987)', '(1992)', '(1941)',
       '(1958)', '(1952)', '(1927)', '(1959)', '(1973)', '(1944)',
       '(1961)', '(2007)', '(1948)', '(2017)', '(2013)', '(2005)',
       '(1976)', '(1989)', '(1982)', '(1939)', '(I) (2020)', '(1965)',
       '(1953)', '(1969)', '(1947)', '(2015)', '(1955)', '(1921)',
       '(1924)', '(1978)', '(1996)', '(I) (20

In [11]:
# extract the year in numbers and convert to data type integer
movie_df['Year of Release'] = movie_df['Year of Release'].str.extract('(\d+)', expand = False).astype('int')
# view first 5 rows
movie_df['Year of Release'][:5]

0    1994
1    1972
2    2008
3    1993
4    2003
Name: Year of Release, dtype: int64

In [12]:
# skim run time column
movie_df['Run Time (minutes)'].unique()

array(['142 min', '175 min', '152 min', '195 min', '201 min', '202 min',
       '96 min', '154 min', '148 min', '178 min', '139 min', '179 min',
       '164 min', '140 min', '169 min', '145 min', '136 min', '133 min',
       '124 min', '157 min', '153 min', '180 min', '127 min', '118 min',
       '189 min', '121 min', '137 min', '125 min', '130 min', '116 min',
       '207 min', '163 min', '155 min', '117 min', '132 min', '151 min',
       '165 min', '106 min', '110 min', '119 min', '88 min', '150 min',
       '109 min', '102 min', '112 min', '89 min', '87 min', '158 min',
       '122 min', '115 min', '120 min', '160 min', '181 min', '113 min',
       '146 min', '147 min', '149 min', '105 min', '98 min', '170 min',
       '95 min', '126 min', '143 min', '141 min', '108 min', '99 min',
       '81 min', '229 min', '131 min', '103 min', '134 min', '218 min',
       '128 min', '129 min', '123 min', '107 min', '161 min', '168 min',
       '135 min', '114 min', '138 min', '111 min', '100 min

In [13]:
# extract only the number in run time and convert to data type integer
movie_df['Run Time (minutes)'] = movie_df['Run Time (minutes)'].str.extract('(\d+)', expand = False).astype('int')
# view first 5 rows
movie_df['Run Time (minutes)'][:5]

0    142
1    175
2    152
3    195
4    201
Name: Run Time (minutes), dtype: int64

In [14]:
# explore rating column
movie_df['Rating'].unique()

array(['R', 'PG-13', 'Approved', 'TV-MA', '', 'PG', 'Not Rated', 'G',
       'Passed', '18+', 'TV-14', '16+', 'M/PG', 'GP', 'TV-PG', 'Unrated',
       'NC-17'], dtype=object)

In [15]:
# convert empty cells to null and convert column to data type string
movie_df['Rating'] = movie_df['Rating'].replace('', np.nan).astype('string')
# view first 5 rows
movie_df['Rating'][:5]

0        R
1        R
2    PG-13
3        R
4    PG-13
Name: Rating, dtype: string

In [16]:
# explore star rating column
movie_df['Star Rating'].unique()

array(['9.3', '9.2', '9.0', '8.9', '8.8', '8.7', '8.6', '8.5', '8.4',
       '8.3', '8.2', '8.1', '8.0', '7.9', '7.8', '7.7', '7.6'],
      dtype=object)

In [17]:
# convert empty cells to null and convert column to data type float
movie_df['Star Rating'] = movie_df['Star Rating'].replace('', np.nan).astype('float')
# view first 5 rows
movie_df['Star Rating'][:5]

0    9.3
1    9.2
2    9.0
3    9.0
4    9.0
Name: Star Rating, dtype: float64

In [18]:
# explore metascore column
movie_df['Metascore'].unique()

array(['82', '100', '84', '95', '94', '90', '97', '74', '92', '67', '87',
       '', '86', '73', '88', '65', '91', '61', '75', '96', '79', '59',
       '89', '98', '85', '66', '81', '64', '62', '77', '57', '80', '99',
       '78', '83', '68', '76', '69', '71', '70', '58', '60', '63', '93',
       '55', '72', '47', '33', '54', '49', '50', '53', '51', '48', '56',
       '44', '45', '52', '46', '30', '28'], dtype=object)

In [19]:
# replace empty cells with null and convert column to data type integer
movie_df['Metascore'] = movie_df['Metascore'].replace('', np.nan).astype('Int64')
# view first 5 rows
movie_df['Metascore'][:5]

0     82
1    100
2     84
3     95
4     94
Name: Metascore, dtype: Int64

In [20]:
# explore number of votes column
movie_df['Number of Votes'].unique()

array(['2,796,722', '1,948,395', '2,777,605', '1,406,133', '1,915,094',
       '1,323,371', '831,117', '2,145,290', '2,465,944', '1,943,254',
       '2,230,612', '2,176,162', '788,749', '1,728,125', '210,145',
       '36,555', '255,835', '1,985,550', '1,213,535', '1,988,714',
       '1,042,756', '1,340,779', '55,248', '120,752', '427,278',
       '1,731,565', '1,492,170', '1,448,536', '1,359,383', '1,412,533',
       '1,140,805', '808,672', '780,966', '721,858', '478,613', '357,689',
       '63,570', '62,762', '1,561,296', '1,395,081', '918,270', '886,179',
       '1,378,624', '1,262,118', '1,633,083', '930,534', '1,208,069',
       '1,157,562', '1,104,516', '1,117,985', '876,556', '697,916',
       '591,393', '897,030', '295,154', '341,390', '508,558', '273,487',
       '252,235', '191,198', '34,040', '1,774,568', '1,398,907',
       '1,012,322', '609,853', '628,419', '416,399', '1,212,870',
       '742,534', '1,286,007', '1,068,309', '691,367', '1,153,900',
       '89,719', '553,332'

In [21]:
# remove commas from column and convert to type integer
movie_df['Number of Votes'] = movie_df['Number of Votes'].str.replace(',', '').astype('int')
# view first 5 rows
movie_df['Number of Votes'][:5]

0    2796722
1    1948395
2    2777605
3    1406133
4    1915094
Name: Number of Votes, dtype: int64

In [22]:
# explore gross column
movie_df['Gross Revenue in millions (USD)'].unique()

array(['$28.34M', '$134.97M', '$534.86M', '$96.90M', '$377.85M',
       '$57.30M', '$4.36M', '$107.93M', '$292.58M', '$315.54M', '$37.03M',
       '$330.25M', '$6.10M', '$342.55M', '#219', '', '#19', '$188.02M',
       '$46.84M', '$171.48M', '$112.00M', '$290.48M', '#41', '$100.13M',
       '$130.74M', '$216.54M', '$136.80M', '$322.74M', '$204.84M',
       '$10.06M', '$7.56M', '$57.60M', '#21', '$0.27M', '#46', '$187.71M',
       '$53.09M', '$78.90M', '$53.37M', '$132.38M', '$210.61M',
       '$162.81M', '$13.09M', '$19.50M', '$6.72M', '$422.78M', '$23.34M',
       '$32.57M', '$32.00M', '$1.02M', '$13.18M', '#45', '$5.32M',
       '$36.76M', '$11.99M', '$0.16M', '$0.02M', '$448.14M', '$335.45M',
       '$248.16M', '$0.71M', '$190.24M', '$51.97M', '$858.37M', '$85.16M',
       '$25.54M', '$44.02M', '$83.47M', '$678.82M', '#91', '$209.73M',
       '$5.02M', '$223.81M', '$11.29M', '$6.53M', '$0.28M', '$1.66M',
       '$11.49M', '#61', '$8.18M', '#62', '$0.29M', '#86', '$718.73M',
       '

In [23]:
# note: some movies did not have gross value and in its place had top 250 ranking starting with # 
# remove values starting with #
movie_df['Gross Revenue in millions (USD)'].mask(movie_df['Gross Revenue in millions (USD)'].str.contains('#'), '', inplace = True)

# remove leading $ and ending M and convert empty cells to null
movie_df['Gross Revenue in millions (USD)'] = movie_df['Gross Revenue in millions (USD)'].str.strip('$M').replace('', np.nan).astype('float')

# show first 5 rows

In [24]:
# inspect cleaned dataframe
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Rank                             1000 non-null   int64  
 1   Movie Title                      1000 non-null   object 
 2   Year of Release                  1000 non-null   int64  
 3   Run Time (minutes)               1000 non-null   int64  
 4   Rating                           985 non-null    string 
 5   Star Rating                      1000 non-null   float64
 6   Metascore                        846 non-null    Int64  
 7   Number of Votes                  1000 non-null   int64  
 8   Gross Revenue in millions (USD)  807 non-null    float64
dtypes: Int64(1), float64(2), int64(4), object(1), string(1)
memory usage: 71.4+ KB


In [25]:
# view descriptive statistics
movie_df.describe().round(3)

,Rank,Year of Release,Run Time (minutes),Star Rating,Metascore,Number of Votes,Gross Revenue in millions (USD)
count,1000.000,1000.000,1000.000,1000.000,846.0,1000.000,807.000
mean,499.501,1991.653,124.155,7.969,79.037,321460.964,70.323
std,288.818,24.203,28.696,0.275,11.977,389994.431,115.474
min,1.000,1920.000,45.000,7.600,28.0,25471.000,0.000
25%,249.750,1975.000,103.000,7.800,71.25,62754.500,3.130
50%,499.500,1999.000,120.000,7.900,80.0,159258.500,23.160
75%,749.250,2011.000,138.250,8.100,88.0,445109.250,83.435
max,999.000,2023.000,321.000,9.300,100.0,2796722.000,936.660


### Step 4: Save dataframe as .csv file

In [26]:
# save file to csv, remove index column
# replace 'path' with location to save file
movie_df.to_csv('path/imdb_top1000.csv', index = False)